In [ ]:
#import libraries
import os
import glob
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models, io
from skimage.io import  imshow, imsave 
from skimage.measure import regionprops_table
import matplotlib
import pandas as pd
import napari
import re


# Final assigment solution - Protein translocation

#  A few words about the  Dataset we will be exploring
Link for downloading the dataset https://bbbc.broadinstitute.org/BBBC014
This 96-well plate has images of cytoplasm to nucleus translocation of the transcription factor NFκB in MCF7 (human breast adenocarcinoma cell line) and A549 (human alveolar basal epithelial) cells in response to TNFα concentration.

For this notebook, we only worked with the MCF7 cells (located in wells A-D)

Images
Images are at 10x objective magnification. The plate was acquired at Vitra Bioscience on the CellCard reader. For each well there is one field with two images: a nuclear counterstain (DAPI) image and a signal stain (FITC) image. Image size is 1360 x 1024 pixels. Images are in 8-bit BMP format.

Recommended citation
"We used image set BBBC014v1 provided by Ilya Ravkin, available from the Broad Bioimage Benchmark Collection [Ljosa et al., Nature Methods, 2012]."

Copyright
Creative Commons LicenseThe SBS Bioimage CNT images are licensed under a Creative Commons Attribution 3.0 Unported License by Ilya Ravkin.

In [ ]:

print(os.getcwd())

In [ ]:
channel2 = plt.imread('BBBC014_v1_images_tiff/Channel 2-01-A-01-00.tiff')

In [ ]:
channel1 = plt.imread('BBBC014_v1_images_tiff/Channel 1-01-A-01-00.tiff')

In [ ]:
imshow(channel1, cmap='gray')

In [ ]:
nuclear_label_folder = 'nuclear_label_folder/'
!mkdir = nuclear_label_folder


In [ ]:
model = models.Cellpose(gpu=True, model_type= 'nuclei')

# apply model

channels = [0,0] # This means we are processing single channel greyscale images.
nuclear_label_image, flows, styles, diams = model.eval(
    channel2, flow_threshold=0.8, diameter=10, channels=channels)


imsave(nuclear_label_folder + '.tiff', label_image.astype('uint16'))
# show result

In [ ]:
plt.imshow(nuclear_label_image)

## Visualize segmentation results using Napari

https://napari.org/stable/

In [ ]:
raw_images_path = Path('BBBC014_v1_images_tiff/')
viewer = napari.Viewer()
viewer.window.resize(1800, 1000)
viewer.open(sorted(raw_images_path.glob("Channel 2-01-A-01-00.tiff")), stack=True)
viewer.open(sorted(raw_images_path.glob("Channel 1-01-A-01-00.tiff")), stack=True)
viewer.add_labels(nuclear_label_image, visible=True)


In [ ]:
from skimage import morphology
import pyclesperanto_prototype as cle 
   # Perform a morphological dilation with clesperanto to make cytoplasm label
cyto_label = cle.dilate_labels(nuclear_label_image, radius=6)
 



        
   
cyto_ring = cyto_label - nuclear_label_image

cyto_ring = np.array(cyto_ring).astype(int)



ring_label_folder = 'ring_label_folder/'
!mkdir = ring_label_folder

imsave(ring_label_folder  + '_label.tiff', cyto_ring.astype('uint16'))

In [ ]:
raw_images_path = Path('BBBC014_v1_images_tiff/')
viewer = napari.Viewer()
viewer.window.resize(1800, 1000)
viewer.open(sorted(raw_images_path.glob("Channel 2-01-A-01-00.tiff")), stack=True)
viewer.open(sorted(raw_images_path.glob("Channel 1-01-A-01-00.tiff")), stack=True)
viewer.add_labels(nuclear_label_image, visible=True)
viewer.add_labels(cyto_ring, visible=True)


In [ ]:
#run morphology and save dafaframe as csv - data wrangling para adquirir a razao
from skimage.measure import regionprops_table

results_ring = regionprops_table(
    cyto_ring, intensity_image= channel1, properties= ('label','mean_intensity'))



results = regionprops_table(nuclear_label_image, intensity_image= channel1, properties= ('label','mean_intensity'))

In [ ]:
#Make dataframe
df_results_ring = pd.DataFrame(results_ring)
df_results_nucleus = pd.DataFrame(results_eroded)


df_results_nucleus['NFKappaB nucleus/cyto']  =  df_results_nucleus['mean_intensity'] / df_results_ring['mean_intensity']

#rename mean_intensity column

df_results_nucleus

In [ ]:
df_results_nucleus['NFKappaB nucleus/cyto'].describe()

In [ ]:
#plot a graph using seaborn
import seaborn as sns
sns.swarmplot(data=df_results_nucleus, y='NFKappaB nucleus/cyto')
